In [2]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import spacy
import re
import string
from tqdm import tqdm
from langdetect import detect, DetectorFactory, LangDetectException
from plotly.subplots import make_subplots
from styles import *

**Preprocesamiento de datos**


In [3]:
features_df = pd.read_csv("data/spotify_music_features.csv")
features_df.head()


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [4]:
lyrics_df = pd.read_csv("data/lyrics_dataset.csv")
lyrics_df.head()

,id,artist,song_name,lyrics
0,4Hhv2vrOTy89HFRcjU3QOx,Etta James,At Last,[Verse 1] At last My love has come along My lo...
1,0zGLlXbHlrAyBN1x6sY0rb,Etta James,A Sunday Kind Of Love,[Verse 1] I want a Sunday kind of love A love ...
2,4ukUoXLuFzMixyZyabSGc4,Ella Fitzgerald,Sleigh Ride,"Just hear those sleigh bells jingling, ring, t..."
3,65irrLqfCMRiO3p87P4C0D,Ella Fitzgerald,Frosty The Snowman,Who's debonair with the tall silk hat Muffler ...
4,2x6pbpjVGjiWCcH89IK8AX,Neil Sedaka,Breaking Up Is Hard to Do,"[Intro] To do do, down dooby doo down down Com..."


In [5]:
# Mergear el datset de feautures con el dataset de letras
df = pd.merge(features_df, lyrics_df, on='id', how='inner')
df.drop(columns=['artist'], inplace=True)

In [6]:
# Analisis general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29100 entries, 0 to 29099
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   valence           29100 non-null  float64
 1   year              29100 non-null  int64  
 2   acousticness      29100 non-null  float64
 3   artists           29100 non-null  object 
 4   danceability      29100 non-null  float64
 5   duration_ms       29100 non-null  int64  
 6   energy            29100 non-null  float64
 7   explicit          29100 non-null  int64  
 8   id                29100 non-null  object 
 9   instrumentalness  29100 non-null  float64
 10  key               29100 non-null  int64  
 11  liveness          29100 non-null  float64
 12  loudness          29100 non-null  float64
 13  mode              29100 non-null  int64  
 14  name              29100 non-null  object 
 15  popularity        29100 non-null  int64  
 16  release_date      29100 non-null  object

In [7]:
df.describe()

,valence,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo
count,29100.000000,29100.000000,29100.000000,29100.000000,2.910000e+04,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000,29100.000000
mean,0.550145,1991.435292,0.326375,0.569475,2.316519e+05,0.589981,0.119141,0.051585,5.244124,0.195065,-9.224761,0.712268,43.797732,0.077527,120.593134
std,0.250507,18.268237,0.316018,0.166139,9.617126e+04,0.237336,0.323960,0.166466,3.533367,0.164259,4.226867,0.452713,15.677694,0.089656,29.717769
min,0.000000,1922.000000,0.000001,0.000000,2.037300e+04,0.000099,0.000000,0.000000,0.000000,0.009670,-36.743000,0.000000,0.000000,0.000000,0.000000
25%,0.348000,1977.000000,0.035300,0.456000,1.813492e+05,0.415000,0.000000,0.000000,2.000000,0.094100,-11.881250,0.000000,33.000000,0.033200,97.126000
50%,0.557000,1993.000000,0.215000,0.576000,2.204080e+05,0.606000,0.000000,0.000020,5.000000,0.130000,-8.560500,1.000000,44.000000,0.043500,118.418000
75%,0.759000,2007.000000,0.595250,0.688000,2.640000e+05,0.785000,0.000000,0.002780,8.000000,0.251000,-5.963000,1.000000,55.000000,0.073700,139.752500
max,1.000000,2020.000000,0.996000,0.978000,5.403500e+06,0.999000,1.000000,0.999000,11.000000,0.997000,1.483000,1.000000,99.000000,0.961000,220.099000


In [8]:
df.isnull().sum()
df.dropna(inplace=True)

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]

In [11]:
# Reemplazar la columna duration_ms por duration_min
df['duration_min'] = (df['duration_ms'] / 60000).round(2)
df.drop(columns=['duration_ms'], inplace=True)


In [12]:
# Crear columna decade
df['decade'] = (df['year'] // 10) * 10
df.drop(columns=['year', 'release_date'], inplace=True)

In [13]:
# Filtrar las canciones de la década de 1960 en adelante
df = df[df['decade'] >= 1960]

In [14]:
# Crear variable binaria de éxito duradero con percentil 80
threshold = df['popularity'].quantile(0.80)
print(f"Umbral de éxito duradero (percentil 80): {threshold:.2f}")

df['was_trending'] = (df['popularity'] >= threshold).astype(int)

Umbral de éxito duradero (percentil 80): 58.00


**Preprocesamiento de letras de canciones**

In [15]:
! python -m spacy download en_core_web_sm
! python -m spacy download es_core_news_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
DetectorFactory.seed = 0
nlp_en = spacy.load("en_core_web_sm", disable=["ner", "parser"])
nlp_es = spacy.load("es_core_news_sm", disable=["ner", "parser"])

In [17]:
def detect_language(text):
    try:
        return detect(text) if isinstance(text, str) and len(text.strip()) > 5 else "unknown"
    except LangDetectException:
        return "unknown"

print("Detectando idiomas ...")
df["language"] = df["lyrics"].apply(detect_language)

Detectando idiomas ...


In [18]:
df["language"].value_counts()

language
en         24612
es          2669
pt           206
fr           146
ja           102
ko            80
it            58
de            47
tl            41
hi            38
id            31
so            29
sw            28
ru            28
tr            26
ro            18
ar            16
he            15
fa            15
nl            14
ca            13
sv             8
pl             8
cy             8
zh-tw          7
unknown        7
vi             7
af             6
et             6
hr             4
no             4
el             4
zh-cn          3
hu             2
sk             2
fi             2
da             2
lv             1
uk             1
bn             1
pa             1
sl             1
th             1
Name: count, dtype: int64

In [19]:
# Filtrar solo las canciones en inglés y español
df = df[df['language'].isin(['en', 'es'])]

In [20]:
# Limpieza general de texto
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()                                     # Minúsculas
    text = re.sub(r'\[.*?\]', '', text)                     # Eliminar [Verse], [Chorus], etc.
    text = re.sub(r'http\S+|www\S+', '', text)              # Eliminar URLs
    text = re.sub(r'\d+', '', text)                         # Eliminar números
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    text = re.sub(r'\s+', ' ', text).strip()                # Quitar espacios extra
    return text

df["lyrics"] = df["lyrics"].apply(clean_text)

In [21]:
# Lematizacion y elimancion de stopwords
def preprocess_text(df):
    results = []

    for lang, nlp in [('en', nlp_en), ('es', nlp_es)]:
        subset_idx = df[df['language'] == lang].index
        subset = df.loc[subset_idx, 'lyrics']

        if subset.empty:
            continue

        texts = [re.sub(r'[^a-záéíóúñü\s]', '', str(t).lower()) for t in subset]
        stopwords = nlp.Defaults.stop_words

        docs = nlp.pipe(texts, batch_size=32, n_process=4)
        tokens_per_doc = []

        for doc in tqdm(docs, total=len(texts), desc=f"Lematizando {lang}"):
            tokens = [
                token.lemma_ for token in doc
                if token.is_alpha and token.lemma_.lower() not in stopwords and len(token) > 2
            ]
            tokens_per_doc.append(tokens)

        results.extend(zip(subset_idx, tokens_per_doc))

    tokens_dict = dict(results)
    df['tokens'] = df.index.map(tokens_dict)
    return df

df = preprocess_text(df)



Lematizando es: 100%|██████████| 2669/2669 [00:54<00:00, 49.12it/s] 


In [22]:
# Detección de valores atípicos
num_cols = [
    'valence', 'acousticness', 'danceability', 
    'duration_min', 'energy', 'instrumentalness'
]

colors = px.colors.qualitative.Plotly

rows, cols = 1, 6
fig = make_subplots(rows=rows, cols=cols)

for i, col in enumerate(num_cols):
    r = (i // cols) + 1
    c = (i % cols) + 1
    fig.add_trace(
        go.Box(
            y=df[col],
            marker_color=colors[i % len(colors)],
            showlegend=True,
            name=col
        ),
        row=r, col=c
    )

fig.update_layout(title='Valores atípicos en características numéricas')
fig = boxplot_style(fig, n_cols=cols, width=1100)
fig.show()

In [23]:
# Detección de valores atípicos
num_cols = [
    'key', 'liveness', 'loudness',
    'popularity', 'speechiness', 'tempo'
]

colors = px.colors.qualitative.Plotly

rows, cols = 1, 6
fig = make_subplots(rows=rows, cols=cols)

for i, col in enumerate(num_cols):
    r = (i // cols) + 1
    c = (i % cols) + 1
    fig.add_trace(
        go.Box(
            y=df[col],
            marker_color=colors[i % len(colors)],
            showlegend=True,
            name=col
        ),
        row=r, col=c
    )

fig.update_layout(title='Valores atípicos en características numéricas')
fig = boxplot_style(fig, n_cols=cols, width=1100)
fig.show()

In [24]:
df.to_csv('data/dataset_processed.csv', index=False)